# `klonowanie repozytorium`

In [1]:
# @title Clone the entire repo { vertical-output: true }
repo_link = "https://github.com/kupieckozienice/pyPDF2" # @param {type:"string"}

# https://github.com/kupieckozienice/pyPDF2

# !git clone -l -s repo_link cloned-repo
# %cd cloned-repo
# !ls

# `instalowanie bibliotek`

In [2]:
# @title install unidecode { form-width: "500px", display-mode: "both" }
%pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.9 MB/s eta 0:00:00


In [3]:
# @title install python-docx { form-width: "750px", display-mode: "both" }
%pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 3.7 MB/s eta 0:00:00


In [4]:
# @title install PyPDF2 { form-width: "600px", display-mode: "both" }
%pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.0 MB/s eta 0:00:00


In [5]:
# @title install pdfminer.six { form-width: "600px", display-mode: "both" }
%pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 42.4 MB/s eta 0:00:00


In [6]:
# @title install openpyxl { form-width: "600px", display-mode: "both" }
%pip install openpyxl

In [7]:
# @title install pillow
%pip install pillow

In [8]:
# @title install faker
%pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.5 MB/s eta 0:00:00


# `import dependencies`

In [9]:
# @title import dependencies { vertical-output: true, form-width: "500px", display-mode: "both" }
# @markdown instalowanie zależności:
# @markdown  `os`,
# @markdown  `pathlib`, `typing`

import os
import faker
import pathlib as pth
from typing import Union, Literal, List
from PyPDF2 import PdfReader, PdfWriter
from pdfminer.high_level import extract_text as fallback_text_extraction
from google.colab import files
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from openpyxl import Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
import openpyxl
from openpyxl.reader.excel import load_workbook
from openpyxl.styles import Alignment, Font
import pandas as pd
from dateutil import tz
import re
from unidecode import unidecode
import random
import string
import time
import random
from datetime import datetime, timedelta

# `data - transformacje daty i czasu`

In [10]:
# @title aktualna data i godzina { vertical-output: true, form-width: "500px", display-mode: "both" }

def actual_datetime():
  teraz = datetime.now(tz=tz.gettz('Europe/Warsaw'))
  formatted_date = teraz.strftime("data_%d_%m_%Y")
  formatted_time = teraz.strftime("czas_%H_%M_%S")

  # print(formatted_date)
  # print(formatted_time)

  return {'data': formatted_date, 'czas': formatted_time}


In [11]:
# @title konwertuje format daty { vertical-output: true, form-width: "500px", display-mode: "both" }
pd = "" # @param {type:"string"}

def convertPdfDatetime(pd):
    dtformat = "%Y%m%d%H%M%S"
    clean = pd.replace("D:","").split('+')[0]
    return datetime.strptime(clean,dtformat)

# `operacje plikowe`

In [36]:
# @title zmień katalog domyślny { vertical-output: true, form-width: "500px", display-mode: "both" }
def zmien_katalog_domyslny(katalog_domyslny):
  os.chdir(katalog_domyslny)
  print(os.getcwd())

# testy
katalog_domyslny = "/content"
zmien_katalog_domyslny(katalog_domyslny)


/content


In [13]:
# @title upload files { vertical-output: true, form-width: "500px", display-mode: "both" }
folder_domyslny = "" # @param {type:"string"}

def upload_files(folder_domyslny = "uploaded"):
  check_if_directory_and_path_exists(folder_domyslny)
  import_files(folder_domyslny)

In [14]:
# @title sprawdzenie czy istnieje katalog roboczy, w którym chcemy pracować { vertical-output: true, form-width: "500px", display-mode: "both" }
directory_path = "" # @param {type:"string"}

def sprawdz_czy_katalog_istnieje(directory_name):
  # Ścieżka do katalogu bieżącego
  cwd = os.getcwd()
  print(f"katalog \"bieżący\": {cwd}")

  working_directory_for_creating = os.path.join(cwd, directory_name)
  print(f"katalog roboczy do utworzenia (jeśli nie istnieje): {working_directory_for_creating}")

  try:
    if not os.path.exists(working_directory_for_creating):
      print(f"katalog: {working_directory_for_creating} nie istnieje i zostanie utworzony")
      os.makedirs(working_directory_for_creating)
    else:
      print(f"katalog: {working_directory_for_creating} już istnieje!")
  except FileExistsError:
    print(f"katalog o nazwie {working_directory_for_creating} już istnieje!")
  except Exception as e:
    print(f"wystąpił błąd podczas tworzenie katalogu: {working_directory_for_creating}")

# testy
# sprawdz_czy_katalog_istnieje("/content/zz")

In [15]:
# @title rozdziel nazwe pliku od rozszerzenia { vertical-output: true, form-width: "500px", display-mode: "both" }
filename = "" # @param {type:"string"}

def rozdziel_nazwe_pliku_od_rozszerzenia(filename):
  nazwa_pliku, rozszerzenie_pliku = os.path.splitext(filename)

  return {'name_key': nazwa_pliku, 'ext_key': rozszerzenie_pliku}

In [16]:
# @title usuń pliki lub katalogi { vertical-output: true, form-width: "500px", display-mode: "both" }
sciezka = "" # @param {type:"string"}
# usuwanie katalogów i plików

import shutil

def usun_pliki_lub_katalogi(sciezka):
  try:
    if os.path.isfile(sciezka):
      os.remove(sciezka)
    elif os.path.isdir(sciezka):
      shutil.rmtree(sciezka)
      print(f"Katalog/Plik '{sciezka}' został usunięty")
  except Exception as e:
    print(f"Wystąpił błąd podczas usuwania '{sciezka}': {e}")



In [17]:
# @title iteruj pliki w podanym katalogu { vertical-output: true, form-width: "500px", display-mode: "both" }

directory_path = "" # @param {type:"string"}

def iterate_over_files_in_directory(path_to_directory):

  numerate = 1

  for filename in os.listdir(path_to_directory):
    f = os.path.join(path_to_directory, filename)
    if os.path.isfile(f):
      print(f"{numerate}. {f}")
      numerate = numerate + 1

In [18]:
# @title iteruje przez podany katalog i wyświetle nazwy plików { vertical-output: true, form-width: "500px", display-mode: "both" }
path_to_directory = "" # @param {type:"string"}

def iterate_over_files_in_directory(path_to_directory):

  numerate = 1

  for filename in os.listdir(path_to_directory):
    f = os.path.join(path_to_directory, filename)
    if os.path.isfile(f):
      print(f"{numerate}. {f}")
      numerate = numerate + 1

In [19]:
# @title sprawdź czy podany plik istnieje { vertical-output: true, form-width: "500px", display-mode: "both" }

path_to_file = "/content/KZP-\u0141A.2110.03.2024 Dostawa skaner\xF3w p\u0142omienia Lamtec.xlsx" # @param {type:"string"}
# @markdown sprawdza czy plik o podanej ścieżce istnieje

def sprawdz_czy_dany_plik_istnieje(path_to_file):
  try:
    with open(path_to_file, 'r') as file:
      print(f"Plik {path_to_file} istnieje i ma się całkiem nieźle ;-)")
      return True
  except FileNotFoundError:
    print('Sorry the file we\'re looking for doesn\' exist')
    return False
  except Exception as e:
    print(f"Wystąpił błąd: {e}")
    return False

# testy
#  /content/KZP-ŁA.2110.03.2024 Dostawa skanerów płomienia Lamtec.xlsx
sprawdz_czy_dany_plik_istnieje("/content/KZP-ŁA.2110.03.2024 Dostawa skanerów płomienia Lamtec.xlsx")

Sorry the file we're looking for doesn' exist


False

In [20]:
# @title check if file exists { vertical-output: true, form-width: "500px", display-mode: "both" }
path_to_file = "" # @param {type:"string"}

def check_if_file_exists(path_to_file):
  try:
    with open(path_to_file, 'r') as file:
      print(f"Plik {path_to_file} istnieje i ma się całkiem nieźle ;-)")
      return True
  except FileNotFoundError:
    print('Sorry the file we\'re looking for doesn\' exist')
    return False
  except Exception as e:
    print(f"Wystąpił błąd: {e}")
    return False

In [21]:
# @title importuj pliki do wskazanego katalogu { vertical-output: true, form-width: "500px", display-mode: "both" }
destination_directory_path = "" # @param {type:"string"}

def import_files(destination_directory_path):
  uploaded = files.upload()
  print(uploaded.keys())

  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

  for filename, file_content in uploaded.items():
    if os.path.exists(destination_directory_path) and os.path.isdir(destination_directory_path):
      with open(os.path.join(destination_directory_path, filename), "wb") as f:
        f.write(file_content)

      # remove files from /content
      file_to_remove_after_copying_to_working_directory = os.path.join(os.getcwd(), filename)
      if os.path.isfile(file_to_remove_after_copying_to_working_directory):
        os.remove(file_to_remove_after_copying_to_working_directory)
        if not os.path.exists(file_to_remove_after_copying_to_working_directory):
          print(f"plik: {os.path.basename(file_to_remove_after_copying_to_working_directory)} "
          f"usunięty z katalogu {os.path.dirname(file_to_remove_after_copying_to_working_directory)}")
        else:
          print(f"plik: {os.path.basename(file_to_remove_after_copying_to_working_directory)} "
          f"NIE usunięty z katalogu {os.path.dirname(file_to_remove_after_copying_to_working_directory)}")

In [22]:
# @title rozdziel nazwe pliku od rozszerzenia { form-width: "500px", display-mode: "both" }
filename = "1_Zalacznik_Zakres_Przedmiotu_zamowienia_KZP_LA_2110_01_2024_Dostawa_silownika_Auma____kopia.xlsx" # @param {type:"string"}

def split_filename_and_extension(filename):
  nazwa_pliku, rozszerzenie_pliku = os.path.splitext(filename)

  return {'name_key': nazwa_pliku, 'ext_key': rozszerzenie_pliku}


# testy

nazwa_pliku_z_rozszerzeniem = split_filename_and_extension(filename)
for key, value in nazwa_pliku_z_rozszerzeniem.items():
  print(nazwa_pliku_z_rozszerzeniem[key])

1_Zalacznik_Zakres_Przedmiotu_zamowienia_KZP_LA_2110_01_2024_Dostawa_silownika_Auma____kopia
.xlsx


In [23]:
# @title zamienia wszystkie inne znaki niż litery i cyfry (oprócz ost. kropki) na '_' (podaj nazwę w formacie nazwa_pliku.rozszerzenie) { form-width: "600px", display-mode: "both" }
nazwa_pliku_do_zmiany_znakow = "KZP-\u0141A.2110.01.2024 Dostawa si\u0142ownika Auma.xlsx" # @param {type:"string"}

# KZP-ŁA.2110.01.2024 Dostawa siłownika Auma.xlsx

def zamien_znaki_w_nazwie_pliku(nazwa_pliku_do_zmiany_znakow):
  nazwa, rozszerzenie = nazwa_pliku_do_zmiany_znakow.rsplit('.',1) if '.' in nazwa_pliku_do_zmiany_znakow else (nazwa_pliku_do_zmiany_znakow, '')
  nazwa_bez_polskich_znakow = unidecode(nazwa)
  zmodyfikowana_nazwa = re.sub(r'[^a-zA-Z0-9]', '_', nazwa_bez_polskich_znakow)
  zmodyfikowany_string = zmodyfikowana_nazwa + ('.' + rozszerzenie if rozszerzenie else '')
  return zmodyfikowany_string

# testy
input_string = r"1.Załącznik Zakres Przedmiotu zamówienia KZP-ŁA.2110.01.2024 Dostawa siłownika Auma — kopia.xlsx"
result = zamien_znaki_w_nazwie_pliku(input_string)
print(result)


1_Zalacznik_Zakres_Przedmiotu_zamowienia_KZP_LA_2110_01_2024_Dostawa_silownika_Auma____kopia.xlsx


In [24]:
# @title sprawdzenie czy istnieje katalog roboczy, w którym chcemy pracować { vertical-output: true, form-width: "500px", display-mode: "both" }
directory_name_path = "" # @param {type:"string"}

def check_if_directory_and_path_exists(directory_name):
  # Ścieżka do katalogu bieżącego
  cwd = os.getcwd()
  print(f"katalog \"bieżący\": {cwd}")

  working_directory_for_creating = os.path.join(cwd, directory_name)
  print(f"katalog roboczy do utworzenia (jeśli nie istnieje): {working_directory_for_creating}")

  try:
    if not os.path.exists(working_directory_for_creating):
      print(f"katalog: {working_directory_for_creating} nie istnieje i zostanie utworzony")
      os.makedirs(working_directory_for_creating)
    else:
      print(f"katalog: {working_directory_for_creating} już istnieje!")
  except FileExistsError:
    print(f"katalog o nazwie {working_directory_for_creating} już istnieje!")
  except Exception as e:
    print(f"wystąpił błąd podczas tworzenie katalogu: {working_directory_for_creating}")

In [25]:
# @title importuj pliki do wskazanego katalogu { vertical-output: true, form-width: "500px", display-mode: "both" }
destination_directory = "" # @param {type:"string"}

def import_files(destination_directory):

  uploaded = files.upload()
  # print(uploaded.keys())

  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

  for filename, file_content in uploaded.items():
    if os.path.exists(destination_directory) and os.path.isdir(destination_directory):
      with open(os.path.join(destination_directory, filename), "wb") as f:
        f.write(file_content)

      # remove files from /content
      file_to_remove_after_copying_to_working_directory = os.path.join(os.getcwd(), filename)
      if os.path.isfile(file_to_remove_after_copying_to_working_directory):
        os.remove(file_to_remove_after_copying_to_working_directory)
        if not os.path.exists(file_to_remove_after_copying_to_working_directory):
          print(f"plik: {os.path.basename(file_to_remove_after_copying_to_working_directory)} "
          f"usunięty z katalogu {os.path.dirname(file_to_remove_after_copying_to_working_directory)}")
        else:
          print(f"plik: {os.path.basename(file_to_remove_after_copying_to_working_directory)} "
          f"NIE usunięty z katalogu {os.path.dirname(file_to_remove_after_copying_to_working_directory)}")

In [26]:
# @title Generator Nazw Plikow { vertical-output: true, form-width: "500px", display-mode: "both" }
class GeneratorNazwPlikow:
  static_variable = 0

  def generuj_nazwe_pliku(self, nazwa_pliku, rozszerzenie = "docx"):
    GeneratorNazwPlikow.static_variable += 1
    file_name = f"{nazwa_pliku}_{GeneratorNazwPlikow.static_variable}.{rozszerzenie}"

    return file_name

In [27]:
# @title czyszczenie zawartości pliku { vertical-output: true, form-width: "500px", display-mode: "both" }

def wyczysc_zawartosc_pliku(sciezka_do_pliku):
  with open(sciezka_do_pliku, "w") as plik:
    plik.truncate(0)

In [28]:
# @title utwórz nowy plik { vertical-output: true, form-width: "500px", display-mode: "both" }

def utworz_nowy_plik(nazwa_pliku):
  aktualna_data_i_godzina = datetime.now().strftime("%d.%m.%Y %H.%M.%S %Z")
  print("aktualna_data_i_godzina:", aktualna_data_i_godzina)
  sciezka_do_pliku = f'/content/{nazwa_pliku}'

  with open(sciezka_do_pliku, "w") as plik:
    plik.write(aktualna_data_i_godzina)


In [29]:
# @title usun_pliki_lub_katalogi { vertical-output: true, form-width: "650px", display-mode: "both" }
usun_pliki_lub_katalogi("/content/NowyFolderDefault")

In [96]:
# @title usun_pliki_lub_katalogi { vertical-output: true, form-width: "650px", display-mode: "both" }
import sys

def zmien_nazwe_katalogu(stara_nazwa, nowa_nazwa):
    try:
      os.rename(stara_nazwa, nowa_nazwa)
      print(f'Zmieniono starą nazwę: {stara_nazwa} na nową: {nowa_nazwa}')
    except OSError as e:
      print(f'Błąd zmiany nazwy pliku/katalogu: {e}')

# `MGR - odczyt plików`

In [97]:
# file_1  = open("/content/uploaded/Arc.txt")

In [98]:
# @title odczyt pliku ver 1 { vertical-output: true, form-width: "600px", display-mode: "both" }

def odczytaj_plik_ver_1(plik):
  file_handler = open(plik, "r")
  read_content = file_handler.read()

  return read_content

In [99]:
# @title odczyt pliku ver 2 { vertical-output: true, form-width: "600px", display-mode: "both" }

def odczytaj_plik_ver_2(plik):
  with open(plik, 'r') as plik:
    przeczytaj_zawartosc = plik.read()

    return przeczytaj_zawartosc

In [100]:
# @title odczyt pliku ver 3 { vertical-output: true, form-width: "600px", display-mode: "both" }

def odczytaj_plik_ver_3(plik):
  try:
    with open(plik, 'r') as plik:
      przeczytaj_zawartosc = plik.read()

      return przeczytaj_zawartosc
  except KeyboardInterrupt:
    print('!! Przerwałeś odczytywanie pliku.')
  finally:
    plik.close()

In [104]:
# @title zapis do pliku { vertical-output: true, form-width: "600px", display-mode: "both" }

# zmien_nazwe_katalogu("/content/uploaded2", "/content/uploaded")

file_writer = open("file_write.txt", "w")
file_writer.write('Programming is Fun.\n')
file_writer.write('Programiz for beginners\n')

file_writer.close()

print(odczytaj_plik_ver_1("file_write.txt"))
print(odczytaj_plik_ver_2("file_write.txt"))
print(odczytaj_plik_ver_3("file_write.txt"))

Programming is Fun.
Programiz for beginners

Programming is Fun.
Programiz for beginners

Programming is Fun.
Programiz for beginners



In [134]:
dict_1 = {'Name': 'Zara', 'Age': 7, 'Class': 'First'}
dict_2 = {'Name': 'Zara', 'Age': 7, 'Class': 'First'}

print("dict['Name']: ", dict_1['Name'])
print("dict['Age']: ", dict_1['Age'])

dict['Name']:  Zara
dict['Age']:  7


In [135]:
# dict['Name'] = "Marta"
# dict['Age'] = 44
# del dict['Name']
# dict.clear()
# print("dict['Name']: ", dict['Name'])
# print("dict['Age']: ", dict['Age'])

In [136]:
dict_3 = dict_1.copy()
dict_3, type(dict_3)

({'Name': 'Zara', 'Age': 7, 'Class': 'First'}, dict)

In [140]:
seq = ('name', 'age', 'sex')
dict_3 = dict_3.fromkeys(seq, 12)
dict_3

{'name': 12, 'age': 12, 'sex': 12}

In [143]:
txt = "Hello, welcome to my world."
x = txt.index("welcome")
y = txt.istitle()
print(x)
print(y)

7
False


In [147]:
txt = "Hello Sam!"
mytable = str.maketrans("S", "P")
print(txt.translate(mytable))

Hello Pam!


In [149]:
txt = "Hi Sam!"
x = "mSa"
y = "eJo"
mytable = str.maketrans(x, y)
print(txt.translate(mytable))

Hi Joe!


In [152]:
tekst = "Arek Łyjak"
x = "AŁ"
y = "SL"
mytable = str.maketrans(x,y)
print(tekst.translate(mytable))

Srek Lyjak


In [154]:
txt = "I could eat bananas all day"

x = txt.partition("bananas")

print(x)

('I could eat ', 'bananas', ' all day')


In [155]:
txt = "I like bananas"

x = txt.replace("bananas", "apples")

print(x)

I like apples


In [156]:
txt = "Mi casa, su casa."

x = txt.rfind("casa")

print(x)

12
